In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
!wget https://learner.csie.ntu.edu.tw/~judge/ml19spring/ml19spring.zip

--2019-06-02 08:24:14--  https://learner.csie.ntu.edu.tw/~judge/ml19spring/ml19spring.zip
Resolving learner.csie.ntu.edu.tw (learner.csie.ntu.edu.tw)... 140.112.90.193
Connecting to learner.csie.ntu.edu.tw (learner.csie.ntu.edu.tw)|140.112.90.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3780741206 (3.5G) [application/zip]
Saving to: ‘ml19spring.zip’

ml19spring.zip      100%[===================>]   3.52G  16.3MB/s    in 5m 8s   

2019-06-02 08:29:24 (11.7 MB/s) - ‘ml19spring.zip’ saved [3780741206/3780741206]



In [3]:
!unzip -q ml19spring.zip
!ls

ml19spring.zip	sample_data  X_test.npz  X_train.npz  Y_train.npz


In [0]:
import numpy as np
x_test = np.load('X_test.npz')['arr_0']
X = np.load('X_train.npz')['arr_0']
Y = np.load('Y_train.npz')['arr_0']

In [0]:
dirPath = './MLP_output/'
!mkdir ./MLP_output

In [0]:
important_fea = np.load('important_feat.npy')
X=X[:,important_fea]
x_test = x_test[:,important_fea]

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
x_train,x_val,y_train,y_val=train_test_split(X,Y,test_size=0.1,random_state=44)
print(x_train.shape,x_val.shape)

Using TensorFlow backend.


(42750, 3752) (4750, 3752)


In [15]:
import pickle
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.models import Model
def baseline_MLP(input_size):
    model = Sequential()
    model.add(Dense(512,input_dim=input_size,kernel_initializer='normal'))
    model.add(PReLU())
    model.add(Dense(128,kernel_initializer='normal'))
    model.add(PReLU())
    model.add(Dense(3,kernel_initializer='normal'))
    model.compile(loss = 'mean_absolute_percentage_error',optimizer='adam')
    return model
def baseline_MLP_WMAE(input_size):
    m_input = Input(shape=(input_size,), dtype='float32')
    X = Dense(512,kernel_initializer='normal')(m_input)
    X = PReLU()(X)
    X = Dense(128,kernel_initializer='normal')(X)
    X = PReLU()(X)
    X1 = Dense(68,kernel_initializer='normal',activation = 'tanh')(X)
    X2 = Dense(68,kernel_initializer='normal',activation = 'tanh')(X)
    X3 = Dense(68,kernel_initializer='normal',activation = 'tanh')(X)
    y1 = Dense(1,kernel_initializer='normal')(X1)
    y2 = Dense(1,kernel_initializer='normal')(X2)
    y3 = Dense(1,kernel_initializer='normal')(X3)
    model = Model(inputs=m_input, outputs=[y1,y2,y3])
    model.compile(optimizer='adam', loss='mean_absolute_error',
              loss_weights=[300,1,200])
    return model
  
dirPath = './MLP_output/' 
model = baseline_MLP_WMAE(x_train.shape[1])
model.summary()

BATCH_SIZE = 64
NUM_EPOCHS = 100

# checkpoint
early_stopping = EarlyStopping(monitor='val_loss',patience=10)
MODELPATH=dirPath+"MLP_model.hdf5"
checkpoint = ModelCheckpoint(MODELPATH, monitor='val_loss', verbose=1, save_best_only=True)

print(y_train.shape)
y_train_T = np.transpose(y_train)
print(y_train_T.shape)
y_val_T = np.transpose(y_val)
print(y_val_T.shape)
history = model.fit(x=x_train,y=[y_train_T[0],y_train_T[1],y_train_T[2]],batch_size=BATCH_SIZE,epochs=NUM_EPOCHS,
        validation_data=(x_val,[y_val_T[0],y_val_T[1],y_val_T[2]]),shuffle=True,callbacks=[early_stopping,checkpoint])
file_his = open(dirPath+'MLP_history.pickle', 'wb')
pickle.dump(history.history, file_his)
file_his.close()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 3752)         0                                            
__________________________________________________________________________________________________
dense_47 (Dense)                (None, 512)          1921536     input_7[0][0]                    
__________________________________________________________________________________________________
p_re_lu_23 (PReLU)              (None, 512)          512         dense_47[0][0]                   
__________________________________________________________________________________________________
dense_48 (Dense)                (None, 128)          65664       p_re_lu_23[0][0]                 
__________________________________________________________________________________________________
p_re_lu_24

In [11]:
if os.path.exists(MODELPATH):
    model.load_weights(MODELPATH)
    # 若成功加载前面保存的参数，输出下列信息
    print("checkpoint_loaded")
    
# checkpoint
early_stopping = EarlyStopping(monitor='val_loss',patience=30)
MODELPATH=dirPath+"MLP_model.hdf5"
checkpoint = ModelCheckpoint(MODELPATH, monitor='val_loss', verbose=1, save_best_only=True)

# print(y_train.shape)
# y_train_T = np.transpose(y_train)
# print(y_train_T.shape)
# y_val_T = np.transpose(y_val)
# print(y_val_T.shape)
history = model.fit(x=x_train,y=[y_train_T[0],y_train_T[1],y_train_T[2]],batch_size=BATCH_SIZE,epochs=NUM_EPOCHS,
        validation_data=(x_val,[y_val_T[0],y_val_T[1],y_val_T[2]]),shuffle=True,callbacks=[early_stopping,checkpoint])
file_his = open(dirPath+'MLP_history.pickle', 'wb')
pickle.dump(history.history, file_his)
file_his.close()

checkpoint_loaded
Train on 42750 samples, validate on 4750 samples
Epoch 1/100
42750/42750 [==============================] - 2s 47us/step - loss: 90.1463 - dense_20_loss: 0.1984 - dense_21_loss: 27.8983 - dense_22_loss: 0.0136 - val_loss: 91.5557 - val_dense_20_loss: 0.2048 - val_dense_21_loss: 27.8636 - val_dense_22_loss: 0.0112

Epoch 00001: val_loss improved from inf to 91.55566, saving model to ./MLP_output/MLP_model.hdf5
Epoch 2/100
42750/42750 [==============================] - 2s 45us/step - loss: 90.9698 - dense_20_loss: 0.1992 - dense_21_loss: 28.1545 - dense_22_loss: 0.0153 - val_loss: 90.0882 - val_dense_20_loss: 0.2015 - val_dense_21_loss: 27.4224 - val_dense_22_loss: 0.0111

Epoch 00002: val_loss improved from 91.55566 to 90.08820, saving model to ./MLP_output/MLP_model.hdf5
Epoch 3/100
42750/42750 [==============================] - 2s 47us/step - loss: 91.0576 - dense_20_loss: 0.1997 - dense_21_loss: 28.2387 - dense_22_loss: 0.0146 - val_loss: 92.9224 - val_dense_20_loss

In [7]:
import os
if os.path.exists(MODELPATH):
    model.load_weights(MODELPATH)
    # 若成功加载前面保存的参数，输出下列信息
    print("checkpoint_loaded")
Y_hat = model.predict(X)

checkpoint_loaded


In [8]:
Y_hat = np.transpose(Y_hat).reshape(-1,3)
err_mat = np.abs(Y-Y_hat)/Y
err_mat = np.sum(err_mat,axis=1)
err_mat = np.sum(err_mat)
print(err_mat/Y.shape[0])
#0.9704514192587647

1.3430223805562302


In [9]:
err_WMAE = np.dot(np.abs(Y-Y_hat),np.array([300,1,200]))
err_WMAE = np.sum(err_WMAE)/Y.shape[0]
print(err_WMAE)
#145.68255323961225

88.47774705235244


In [0]:
Y_test_hat = model.predict(x_test)
Y_test_hat = np.transpose(Y_test_hat).reshape(-1,3)
np.savetxt(dirPath+'MLP_Multioutput_1.csv',Y_test_hat,delimiter=',')

In [22]:
# Save model to your Drive
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [26]:
!zip MLP_output MLP_output/*

  adding: MLP_output/MLP.csv (deflated 61%)
  adding: MLP_output/MLP_history.pickle (deflated 41%)
  adding: MLP_output/MLP_model.hdf5 (deflated 43%)


In [0]:
!mv MLP_output.zip '/gdrive/My Drive/term_2019_1/MLTech/Final'